In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox

In [ ]:
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

In [ ]:
city = ox.geocode_to_gdf("Pune City,Pune District,Maharashtra,India")
print(city.crs)
city = ox.project_gdf(city)
fig, ax = ox.plot.plot_footprints(city)
print(city.crs)

In [ ]:
city.to_file('PuneFiles/Pune_city.shp')

In [ ]:
geometry = city['geometry'].iloc[0]
print(type(geometry))

In [ ]:
geometry_cut = ox.utils_geo._quadrat_cut_geometry(geometry,840)
# geometry_cut = ox.quadrat_cut_geometry(geometry, 840)  # Syntax for new version
print(type(geometry_cut))

In [ ]:
polylist = [p for p in geometry_cut] 
polylist[5]

In [ ]:
polylist = [p for p in geometry_cut]

# plot city
west, south, east, north = city.unary_union.bounds

fig, ax = plt.subplots(figsize=(40,40))
for polygon, n in zip(geometry_cut, np.arange(len(polylist))):
    p = polygon.representative_point().coords[:][0]   #list/tuple of centre coordinates of each block 
    patch = PolygonPatch(polygon, fc='#ffffff', ec='#000000', alpha=0.5, zorder=2)
    ax.add_patch(patch)
    plt.annotate(text=n, xy=p, horizontalalignment='left', size=15, va='bottom')
    
ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

In [ ]:
polyframe = gpd.GeoDataFrame(geometry=polylist)
polyframe.crs = city.crs
print(polyframe.crs)
polyframe.head()

In [ ]:
len(polyframe)

In [ ]:
import contextily as ctx
# ctx uses epsg:3857
polyframe_3857 = polyframe.to_crs(epsg=3857)
west, south, east, north = polyframe_3857.unary_union.bounds

ax = polyframe_3857.plot(figsize=(40,40), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=13)
ax.set_xlim(west, east)
ax.set_ylim(south, north)

In [ ]:
l = len(polylist)
OD_matrix = np.zeros(shape=(l,l))
print(OD_matrix)
print(l)

In [ ]:
# Population Data
from random import randint
row=randint(2,80)
opd=pd.read_csv("PunePop.csv")
pune_pop1=opd.iloc[row]
print(pune_pop1)

In [ ]:
pune_pop1=pune_pop1.to_numpy()
type(pune_pop1)

In [ ]:
pune_pop1=np.append(pune_pop1,[1500,1500])

In [ ]:
plt.hist(pune_pop1, bins=50)
plt.show()

In [ ]:
polyframe.head()

In [ ]:
print('Pune Geodataframe Length: ', len(polyframe))
print('Pune Population Array Length: ', len(pune_pop1))

In [ ]:
type(pune_pop1)

In [ ]:
polyframe['population'] = pune_pop1
polyframe.head()

In [ ]:
polylist = [p for p in polyframe_3857.geometry]
# 3720
# plot city
west, south, east, north = polyframe_3857.unary_union.bounds

ax = polyframe_3857.plot(figsize=(40,40), alpha=0.5, edgecolor='k', zorder=5)
for polygon, n in zip(polylist, np.arange(len(polylist))):
    p = polygon.representative_point().coords[:][0]
    patch = PolygonPatch(polygon, fc='#ffffff', ec='#000000', alpha=0.5, zorder=2)
    ax.add_patch(patch)
    plt.annotate(s=n, xy=p, horizontalalignment='center', size=15)
ctx.add_basemap(ax, zoom=13)
    
ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

In [ ]:
plt.rcParams.update({'font.size':32})
polyframe_3857 = polyframe.to_crs(epsg=3857)
west, south, east, north = polyframe_3857.unary_union.bounds

fig, ax = plt.subplots(figsize=(40,40))
polyframe_3857.plot(ax=ax, column = 'population', legend=False, cmap='rainbow')

cbax = fig.add_axes([0.915, 0.175, 0.02, 0.7])

sm = plt.cm.ScalarMappable(cmap='rainbow', \
                          norm = plt.Normalize(vmin=min(polyframe_3857.population), vmax=max(polyframe_3857.population)))

sm._A = []

# draw colormap into cbax

fig.colorbar(sm, cax=cbax, format="%d")

ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()